In [1]:
import pandas as pd

from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.live.nba.endpoints import boxscore

In [2]:
# Data Sets for
# box = boxscore.BoxScore(game_id)
# ---------------------------------
#box.game.get_dict()                    #equal to box.get_dict()['game']
#box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
#box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
#box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
#box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
#box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
#box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
#box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

In [3]:
def boxpuller(homeTeam, date):
    #Get the team_id
    nba_teams = teams.get_teams()
    # Select the dictionary for the Pacers, which contains their team ID
    teamA = [team for team in nba_teams if team['abbreviation'] == homeTeam][0]
    teamA_id = teamA['id']

   # Query for the last regular season game where teamA were playing


    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=teamA_id,date_from_nullable=date,
    date_to_nullable=date)  
    gamess = gamefinder.get_data_frames()[0].sort_values('GAME_DATE').iloc[-1]
    game_id = gamess.GAME_ID
    matchup = gamess.MATCHUP

    print(f'Getting boxscore for {matchup} on {date}')
    
    # Get **all** the games so we can filter to an individual GAME_ID
    result = leaguegamefinder.LeagueGameFinder()
    all_games = result.get_data_frames()[0]
    # Find the game_id we want
    full_game = all_games[all_games.GAME_ID == game_id]

    return full_game

In [4]:
box = boxpuller('OKC', '03/21/2023')
box

Getting boxscore for OKC @ LAC on 03/21/2023


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
3162,22022,1610612760,OKC,Oklahoma City Thunder,0022201082,2023-03-21,OKC @ LAC,W,239,101,...,0.929,12,35,47,19,2,4,11,21,1.0
3177,22022,1610612746,LAC,LA Clippers,0022201082,2023-03-21,LAC vs. OKC,L,239,100,...,0.571,12,34,46,21,9,2,13,21,-1.0


In [5]:
def boxpuller(homeTeam, date):
    #Get the team_id
    nba_teams = teams.get_teams()
    # Select the dictionary for the Pacers, which contains their team ID
    teamA = [team for team in nba_teams if team['abbreviation'] == homeTeam][0]
    teamA_id = teamA['id']

    # Query for the last regular season game where teamA were playing


    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=teamA_id)  

    games = gamefinder.get_data_frames()[0]
    gamess = games[games.GAME_DATE == date].sort_values('GAME_DATE').iloc[-1]
    game_id = gamess.GAME_ID
    matchup = gamess.MATCHUP
    awayTeam = gamess.MATCHUP.split()[2]
    print(f'Getting boxscore for {matchup} on {date}')
    
    box = boxscore.BoxScore(game_id)
    
    all_player_stats = []
    
    ht_stats = box.home_team_player_stats.get_dict()
    for player in ht_stats:
        if player['status'] == 'ACTIVE':
            player_stats = player['statistics']
            player_stats.update({"NAME": player['name'],"TEAM":homeTeam, "OPP":awayTeam, "DATE":date})
            all_player_stats.append(player_stats)
            
    at_stats = box.away_team_player_stats.get_dict()
    for player in at_stats:
        if player['status'] == 'ACTIVE':
            player_stats = player['statistics']
            player_stats.update({"NAME": player['name'],"TEAM":awayTeam, "OPP":homeTeam, "DATE":date})
            all_player_stats.append(player_stats)
                
    
    return all_player_stats

In [6]:
gameLog = [('PHI','2009-02-07'),
          ('WAS','2017-02-13'),
          ('OKC','2017-04-04'),
          ('DEN','2017-04-09'),
          ('GSW','2018-02-24'),
          ('SAC','2018-11-19'),
          ('GSW','2018-11-21')]

gameLog2 = [
          ('GSW','2019-11-25'),
          ('SAC','2019-12-11'),
          ('BKN','2020-01-07'),
          ('HOU','2020-01-15'),
          ('HOU','2020-01-20'),
          ('PHI','2021-04-26'),
          ('WAS','2021-05-29'),
          ('LAC','2021-11-01'),
          ('LAL','2021-11-02'),
          ('LAL','2021-11-04'),
          ('OKC','2021-11-12'),
          ('LAL','2023-02-07'),
          ('PHX','2023-02-24'),
          ('LAC','2023-03-18'),
          ('LAC','2023-03-21')]

gameLogPlayerStats = []

for team,date in gameLog2:
    print(team,date)
    gamePlayerStats = boxpuller(team,date)
    for playerStats in gamePlayerStats:
        gameLogPlayerStats.append(playerStats)
    
df = pd.DataFrame(gameLogPlayerStats)
df = df.drop(columns=['blocksReceived', 'foulsOffensive','foulsDrawn','minus',
                    'plus','plusMinusPoints','pointsFastBreak','pointsInThePaint','pointsSecondChance',
                         'twoPointersAttempted','twoPointersMade','twoPointersPercentage',
                     'threePointersPercentage','freeThrowsPercentage'])
    
df = df[['NAME','TEAM','DATE','OPP', 'minutesCalculated', 'points','reboundsTotal','assists','steals','blocks',
 'fieldGoalsMade', 'fieldGoalsAttempted', 'fieldGoalsPercentage',
 'threePointersMade', 'threePointersAttempted',
 'freeThrowsMade', 'freeThrowsAttempted',
 'reboundsDefensive','reboundsOffensive',
 'turnovers',
 'foulsPersonal', 'foulsTechnical']]

df = df.rename(columns={"minutesCalculated": "MINS", "points": "PTS", "reboundsTotal": "REB",
                       "assists": "AST","steals": "STL","blocks": "BLK",
                        "fieldGoalsMade": "FGM", "fieldGoalsAttempted": "FGA", "fieldGoalsPercentage": "FG%",
                       "threePointersMade": "3PM", "threePointersAttempted": "3PA",
                       "freeThrowsMade": "FTM", "freeThrowsAttempted": "FTA",
                        "reboundsDefensive":"dREB", "reboundsOffensive":"oREB","turnovers":"TO",
                       "foulsPersonal":"FLS", "foulsTechnical":"TFL"})

df = df.round({'FG%': 2})
df['MINS'] = df['MINS'].str.replace(r'PT', '')
df['MINS'] = df['MINS'].str.replace(r'M', '')
df

GSW 2019-11-25
Getting boxscore for GSW vs. OKC on 2019-11-25
SAC 2019-12-11
Getting boxscore for SAC vs. OKC on 2019-12-11
BKN 2020-01-07
Getting boxscore for BKN vs. OKC on 2020-01-07
HOU 2020-01-15
Getting boxscore for HOU vs. POR on 2020-01-15
HOU 2020-01-20
Getting boxscore for HOU vs. OKC on 2020-01-20
PHI 2021-04-26
Getting boxscore for PHI vs. OKC on 2021-04-26
WAS 2021-05-29
Getting boxscore for WAS vs. PHI on 2021-05-29
LAC 2021-11-01
Getting boxscore for LAC vs. OKC on 2021-11-01
LAL 2021-11-02
Getting boxscore for LAL vs. HOU on 2021-11-02
LAL 2021-11-04
Getting boxscore for LAL vs. OKC on 2021-11-04
OKC 2021-11-12
Getting boxscore for OKC vs. SAC on 2021-11-12
LAL 2023-02-07
Getting boxscore for LAL vs. OKC on 2023-02-07
PHX 2023-02-24
Getting boxscore for PHX vs. OKC on 2023-02-24
LAC 2023-03-18
Getting boxscore for LAC vs. ORL on 2023-03-18
LAC 2023-03-21
Getting boxscore for LAC vs. OKC on 2023-03-21


,NAME,TEAM,DATE,OPP,MINS,PTS,REB,AST,STL,BLK,...,FG%,3PM,3PA,FTM,FTA,dREB,oREB,TO,FLS,TFL
0,Glenn Robinson III,GSW,2019-11-25,OKC,36,25,2,5,2,1,...,64.29,3,4,4,5,1,1,1,1,0
1,Eric Paschall,GSW,2019-11-25,OKC,32,13,10,1,0,1,...,66.67,0,0,1,1,8,2,5,5,0
2,Willie Cauley-Stein,GSW,2019-11-25,OKC,29,4,6,1,2,0,...,28.57,0,0,0,0,4,2,0,3,0
3,Alec Burks,GSW,2019-11-25,OKC,33,7,4,3,1,0,...,17.65,1,4,0,0,4,0,3,1,0
4,Ky Bowman,GSW,2019-11-25,OKC,39,24,3,5,3,1,...,52.94,2,3,4,4,1,2,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,Ousmane Dieng,OKC,2023-03-21,LAC,08,3,1,1,0,0,...,0.25,1,3,0,0,1,0,0,0,0
378,Aaron Wiggins,OKC,2023-03-21,LAC,20,2,6,1,0,0,...,0.33,0,0,0,0,5,1,0,2,0
379,Tre Mann,OKC,2023-03-21,LAC,00,0,0,0,0,0,...,0.00,0,0,0,0,0,0,0,0,0
380,Jeremiah Robinson-Earl,OKC,2023-03-21,LAC,00,0,0,0,0,0,...,0.00,0,0,0,0,0,0,0,0,0


In [7]:
df.sort_values(by=['oREB'],ascending=False).head(10)

,NAME,TEAM,DATE,OPP,MINS,PTS,REB,AST,STL,BLK,...,FG%,3PM,3PA,FTM,FTA,dREB,oREB,TO,FLS,TFL
332,Ivica Zubac,LAC,2023-03-18,ORL,27,16,16,0,0,2,...,0.44,0,0,2,2,9,7,2,2,0
86,Hassan Whiteside,POR,2020-01-15,HOU,37,18,18,0,0,2,...,0.44,0,0,4,8,11,7,1,4,0
74,Clint Capela,HOU,2020-01-15,POR,36,14,18,2,1,1,...,0.50,0,0,0,0,11,7,3,2,0
225,Anthony Davis,LAL,2021-11-04,OKC,38,29,18,5,1,2,...,0.55,0,1,7,9,12,6,3,4,0
204,Russell Westbrook,LAL,2021-11-02,HOU,37,27,9,7,1,0,...,0.48,0,4,7,8,4,5,3,1,0
130,Paul Reed,PHI,2021-04-26,OKC,10,10,9,1,1,2,...,0.71,0,0,0,0,4,5,1,0,0
311,Jock Landale,PHX,2023-02-24,OKC,17,5,7,1,1,0,...,0.20,0,1,3,3,2,5,0,0,0
39,Steven Adams,OKC,2019-12-11,SAC,28,12,10,4,0,2,...,0.50,0,0,2,5,5,5,1,1,0
152,Daniel Gafford,WAS,2021-05-29,PHI,23,16,6,0,1,1,...,0.75,0,0,4,4,1,5,0,3,0
109,Danilo Gallinari,OKC,2020-01-20,HOU,35,25,13,0,1,0,...,0.37,4,8,7,7,8,5,4,1,0


In [8]:
display(df.loc[(df['PTS']>=10) & (df['REB'] >= 10) & (df['AST']>=10)])

,NAME,TEAM,DATE,OPP,MINS,PTS,REB,AST,STL,BLK,...,FG%,3PM,3PA,FTM,FTA,dREB,oREB,TO,FLS,TFL
76,Russell Westbrook,HOU,2020-01-15,POR,42,31,11,12,1,0,...,0.50,2,5,7,12,10,1,1,1,1
100,Russell Westbrook,HOU,2020-01-20,OKC,38,32,11,12,2,0,...,0.67,0,1,0,1,9,2,3,0,0
150,Russell Westbrook,WAS,2021-05-29,PHI,34,26,12,10,0,0,...,0.50,3,6,5,5,9,3,3,4,0
